In [1]:
# !pip install -q pycaret

In [2]:
# --- 1. 라이브러리 로딩 ---
import os
import pandas as pd
import numpy as np
from pycaret.regression import *

In [ ]:
# --- 2. 데이터 로딩 및 전처리 ---
print("\n--- 2. 데이터 로딩 및 전처리 ---")
DATA_FOLDER = "./"
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError(f"'{DATA_FOLDER}' 폴더에 분석할 CSV 파일이 없습니다.")

df_list = []
for csv_file in csv_files:
    file_path = os.path.join(DATA_FOLDER, csv_file)
    df_temp = pd.read_csv(file_path)
    df_list.append(df_temp)

df_raw = pd.concat(df_list, ignore_index=True).sample(frac=0.01, random_state=42).reset_index(drop=True)
print(f"데이터 로딩 완료. 총 {len(df_raw)}개 행.")

df_raw['시'] = df_raw['시'].replace(24, 0)
df_raw['datetime'] = pd.to_datetime(
    df_raw['연도'].astype(str) + '-' +
    df_raw['월'].astype(str).str.zfill(2) + '-' +
    df_raw['일'].astype(str).str.zfill(2) + ' ' +
    df_raw['시'].astype(str).str.zfill(2) + ':00:00',
    format='%Y-%m-%d %H:%M:%S'
)

df_processed = df_raw.sort_values('datetime').set_index('datetime')
df_processed = df_processed.groupby(df_processed.index).mean()

TARGET_COLUMN = '전력부하합계'
feature_columns = ['기온', '상대습도', TARGET_COLUMN]
df_processed = df_processed[feature_columns]

print("결측치 처리 전:\n", df_processed.isnull().sum().loc[lambda x: x > 0])
df_processed.fillna(method='ffill', inplace=True)
df_processed.fillna(method='bfill', inplace=True)
print("\n결측치 처리 완료.")

print("사용자 정의 특성(불쾌지수) 생성 중...")
df_processed['불쾌지수'] = 1.8 * df_processed['기온'] - 0.55 * (1 - df_processed['상대습도']/100) * (1.8 * df_processed['기온'] - 26) + 32

df_final = df_processed.copy()

# datetime 정보를 피처로 분해
df_final['year'] = df_final.index.year
df_final['month'] = df_final.index.month
df_final['day'] = df_final.index.day
df_final['hour'] = df_final.index.hour
df_final['dayofweek'] = df_final.index.dayofweek

print(f"데이터 준비 완료. {len(df_final)}개 행.")
print("\n최종 데이터 샘플:")
print(df_final.head())

In [ ]:
# --- 3. PyCaret Regression 환경 설정 ---
print("\n--- 3. PyCaret 회귀 환경 설정 ---")
s = setup(
    data=df_final.reset_index(drop=True),
    target=TARGET_COLUMN,
    session_id=42,
    normalize=True,
    fold=3,
    verbose=False
)

# --- 4. 모델 비교 및 최적 모델 선택 ---
print("\n--- 4. 모델 비교 중 (compare_models) ---")
best = compare_models()

In [ ]:
# --- 5. 모델 튜닝 ---
print("\n--- 5. 모델 튜닝 중 (tune_model) ---")
tuned = tune_model(best, optimize='MAE')

In [ ]:
# --- 6. 시각화 및 분석 ---
print("\n--- 6. 모델 시각화 (plot_model) ---")
plot_model(tuned, plot='residuals')
plot_model(tuned, plot='error')
plot_model(tuned, plot='feature')

In [ ]:
# --- 7. 예측 및 결과 확인 ---
print("\n--- 7. 테스트셋 예측 결과 ---")
pred = predict_model(tuned)
pred.head()

In [ ]:
# --- 8. 모델 저장 및 재사용 ---
save_model(tuned, 'final_powerload_model')
print("\n모델 저장 완료: final_powerload_model.pkl")